In [ ]:
# Don't change the names
!mkdir msdir
!mkdir input
!mkdir output

In [ ]:
import os
import json
import stimela
import pkg_resources
from drasticali import drasticali
from drasticali.drasticali import natural_keys, remove_cab
from ipywidgets import widgets, interact
from IPython.display import Javascript

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)

docker = widgets.Dropdown(options=['Docker', 'uDocker', 'Podman', 'Singularity'],
    value='uDocker',
    description='Container Tech'
)
display(docker)

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
DATA = dict()
MSDIR = 'msdir'
if docker.value in ['Docker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='docker')
elif docker.value in ['uDocker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='udocker')
elif docker.value in ['Podman']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='podman')
elif docker.value in ['Singularity']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR,
                            singularity_image_dir=os.environ["SINGULARITY_PULLFOLDER"])
cab_names = ['simms', 'simulator', 'calibrator', 'wsclean', 'ddfacet', 'cleanmask', 'casa_makemask',
             'casa_clean', 'lwimager', 'casa_tclean',
             'fitstool', 'pybdsm', 'aegean',]#stimela.CAB
def setup():
    for cab_name in cab_names:
        PARAM_FILE_PATH = pkg_resources.resource_filename('stimela', 'cargo/cab/{}'.format(cab_name))
        PARAM_FILE = '{}/parameters.json'.format(PARAM_FILE_PATH)

        with open(PARAM_FILE) as par:
            DATA[cab_name] = json.load(par)
setup()

In [ ]:
recipe_file = widgets.Text(
    value='',
    placeholder='recipe.json',
    description='Recipe File:',
    disabled=False
)
display(recipe_file)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="LOAD")
button.on_click(run_all)
display(button)

Select cab to use:

In [ ]:
cab = widgets.Dropdown(
    options=cab_names,
    description='Task:',
    disabled=False)
display(cab)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="SELECT")
button.on_click(run_all)
display(button)

In [ ]:
c = DATA[cab.value]
print(c['description'])
print('')
params = c['parameters']
PAR = dict()
for param in params:
    #PAR = dict()
    print("Name: " + str(param['name']))
    print("Info: " + str(param['info']))
    print("Type: " + str(param['dtype']))
    if 'default' in param:
        print "Default: " + str(param['default'])
    if 'required' in param:
        print "Required: " + str(param['required'])
    if param['dtype'] in ['str', 'file', 'list:str', #'list:int',
                          'list:file', 'list:float', 'list:list'] or param['default'] == None:
        PAR[param['name']] = widgets.Text(value=str(param['default']) if 'default' in param else '')
    elif param['dtype'] in ['list:int'] and param['default'] != None:
        try:
            PAR[param['name']] = widgets.IntText(value=param['default'] if 'default' in param else None)
        except :
            try:
                PAR[param['name']] = widgets.Textarea(value=param['default'] if 'default' in param else None)
            except:
                PAR[param['name']] = widgets.Text(value=str(param['default']) if 'default' in param else None)
    elif type(param['dtype']) is list:
        PAR[param['name']] = widgets.Textarea(value=str(param['default']) if 'default' in param else '')
    elif param['dtype'] in ['int'] and param['default'] != None:
        PAR[param['name']] = widgets.IntText(value=param['default'] if 'default' in param else None)
    elif param['dtype'] in ['float'] and param['default'] != None:
        PAR[param['name']] = widgets.FloatText(value=float(param['default']) if 'default' in param else None)
    elif param['dtype'] in ['bool']:
        PAR[param['name']] = widgets.Dropdown(options=[True, False],
                                              value=bool(param['default']))# if 'default' in param else '')
    display(PAR[param['name']])
    print('')

In [ ]:
cab_number = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Cab Number:',
    disabled=False
)
print("Note: 0 means append at the end")
display(cab_number)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="ADD")
button.on_click(run_all)
display(button)

cab_params = {}

In [ ]:
cab_num = cab_number.value
cab_func_name = cab.value
cab_func = getattr(drasticali, 'cab_function', None)
for key, val in PAR.items():
    if val.value in ['None']:
        continue
    if type(val.value) is unicode and '\\n' in val.value:
        cab_params[key] = val.value.split('\\n')
    ##temp fix since the type must be int
    elif key in ['sigma']:
        cab_params[key] = int(val.value)
    else:
        for param in params:
            if param['dtype'] in ['int'] and param['name'] == key:
                cab_params[key] = int(val.value)
                continue
            elif param['dtype'] in ['float'] and param['name'] == key:
                cab_params[key] = float(val.value)
                continue
            elif type(param['dtype']) is list and param['name'] == key:
                for p in param['dtype']:
                    try:
                        if p in ['int']:
                            cab_params[key] = int(val.value)
                        #elif p in ['float']:
                        #    cab_params[key] = float(val.value)
                        elif type(val.value) is unicode:
                            cab_params[key] = str(val.value)
                    except ValueError:
                        cab_params[key] = str(val.value)
                continue
            elif type(val.value) is unicode and param['name'] == key:
                cab_params[key] = str(val.value)
                continue
            else:
                if param['name'] == key:
                    cab_params[key] = val.value

cab_func(recipe, name=cab_func_name, num=cab_num, parameters=cab_params,
         recipes_file=recipe_file.value, dump=False)

drasticali.run_all(recipe, recipe_file.value)

In [ ]:
#%load_ext jupyternotify

def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="RUN ALL")
button.on_click(run_all)
display(button)

In [ ]:
drasticali.run_all(recipe, recipe_file.value)

## Remove Unwanted cabs

In [ ]:
data = drasticali.get_data(recipe_file.value)
recipe.jobs = []
data_keys = data.keys()
data_keys.sort(key=natural_keys)
cab1 = widgets.Dropdown(
    options=data_keys,
    description='Task:',
    disabled=False)
display(cab1)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+2)'))

button = widgets.Button(description="REMOVE")
button.on_click(run_all)
display(button)

In [ ]:
remove_cab(cab1.value, recipe_file.value)